[View in Colaboratory](https://colab.research.google.com/github/znah/lucid/blob/master/notebooks/differentiable-parameterizations/texture_synth_3d.ipynb)

## Install and import

In [0]:
!pip -q install git+https://github.com/tensorflow/lucid.git

In [0]:
import os
import io
import sys
from string import Template

import numpy as np
import PIL.Image
import matplotlib.pylab as pl

import tensorflow as tf

from IPython.display import clear_output, display, Image, HTML

from lucid.misc.gl.glcontext import create_opengl_context
import OpenGL.GL as gl

from lucid.misc.gl import meshutil
from lucid.misc.gl import glrenderer
import lucid.misc.io.showing as show
from lucid.misc.io import load
from lucid.misc.tfutil import create_session

from lucid.modelzoo import vision_models
from lucid.optvis import objectives
from lucid.optvis import param
from lucid.optvis.param.spatial import sample_bilinear

In [0]:
create_opengl_context()
print(gl.glGetString(gl.GL_VERSION))

## Loading 3D model

In [0]:
!gsutil cp gs://deepdream/article_models.zip . && unzip -qo article_models.zip
!cat article_models/readme.txt

In [0]:
mesh = meshutil.load_obj('article_models/bunny.obj')
mesh = meshutil.normalize_mesh(mesh)
original_texture = load('article_models/bunny.png')

## Texture Synthesis

In [0]:
renderer = glrenderer.MeshRenderer((512, 512))

In [0]:
model = vision_models.InceptionV1()
model.load_graphdef()

In [0]:
sess = create_session()

t_fragments = tf.placeholder(tf.float32, [None, None, 4]) # channels: UV_A
t_uv = t_fragments[...,:2]
t_alpha = t_fragments[...,3:]

t_texture = param.image(1024, fft=True, decorrelate=True)[0]
t_frame = sample_bilinear(t_texture, t_uv) * t_alpha

model.import_graph(t_frame)


def T(layer):
  return sess.graph.get_tensor_by_name("import/%s:0"%layer)

obj = objectives.as_objective('mixed4b_pool_reduce_pre_relu:17')(T)
tf.losses.add_loss(-obj)


t_lr = tf.constant(0.01)
t_loss = tf.losses.get_total_loss()
trainer = tf.train.AdamOptimizer(t_lr)
train_op = trainer.minimize(t_loss)

init_op = tf.global_variables_initializer()
init_op.run()


In [0]:
# Verify that model renders correctly
fragments = renderer.render_mesh(modelview=meshutil.sample_view(11.0, 13.0), **mesh)
img = t_frame.eval({t_fragments: fragments, t_texture: original_texture})
show.image(img, 'jpeg')

In [0]:
loss_log = []
init_op.run()

In [0]:
for i in range(400):
  # Render mesh UVs with OpenGL
  fragments = renderer.render_mesh(modelview=meshutil.sample_view(11.0, 13.0), **mesh)
  # Perform step optimization for the current view
  _, loss = sess.run([train_op, t_loss], {t_fragments: fragments, t_lr:0.03})
  loss_log.append(loss)
  if i==0 or (i+1)%50 == 0:
    clear_output()
    last_frame = sess.run(t_frame, {t_fragments: fragments})
    show.images([last_frame, fragments], ['current', 'uv'])
  if i==0 or (i+1)%10 == 0:
    print(len(loss_log), loss)


In [0]:
pl.plot(loss_log);

## Display Result

In [0]:
texture = t_texture.eval()
show.textured_mesh(mesh, texture)

In [0]:
show.image(texture, 'jpeg')